# 4. Entraînement du modèle sur les images sélectionnées

Préparation de la configuration de YOLO

Il faut cloner le repo yolov5 et installer les packages nécessaires. Pour faciliter l'entraînement, il est plus pratique de travailler dans le repo

```bash
git clone https://github.com/ultralytics/yolov5.git  # clone
cd yolov5
pip install -r requirements.txt  # install
```

In [2]:
import os
import random
import shutil

Préparation des jdds train et val dans le dossier yolov5/modele et de la structure

In [12]:
# Structure
model_path = "../yolov5/modele/"
os.makedirs(model_path, exist_ok=True)

for dir in ["train", "valid"]:
    for subdir in ["images", "labels"]:
        os.makedirs(os.path.join(model_path, dir, subdir), exist_ok=True)

In [4]:
# Pour obtenir les mêmes distributions
random.seed(1234)
# Récupérer les noms d'image annotées
annotated_images_names = os.listdir("../data/annotations/images/")
# Mélanger les noms
random.shuffle(annotated_images_names)

# Utiliser un split 85/15
split_index = int(0.85 * len(annotated_images_names))
train_images = annotated_images_names[:split_index]
valid_images = annotated_images_names[split_index:]

# Copier les fichiers dans le dossier yolov5
train_dir = "../yolov5/modele/train/"
valid_dir = "../yolov5/modele/valid/"

# Copier selon les noms dans le bon dossier
def copy_images(image_list, source_dir, target_dir):
    for image_name in image_list:
        # Définir le nom du label
        lab_name = image_name.replace("jpg", "txt")

        # Définir les chemins d'origine
        im_path = os.path.join(source_dir, "images", image_name)
        lab_path = os.path.join(source_dir, "labels", lab_name)

        # Définir les chemins de destination
        im_target_path = os.path.join(target_dir, "images", image_name)
        lab_target_path = os.path.join(target_dir, "labels", lab_name)

        # Copier les images
        shutil.copy(im_path, im_target_path)
        shutil.copy(lab_path, lab_target_path)

# Copier les images
copy_images(train_images, "../data/annotations", train_dir)
copy_images(valid_images, "../data/annotations", valid_dir)

Statistiques du nombre de piscines dans chaque set (1 ligne par objet)

In [5]:
def count_lines_in_file(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file)

def count_lines_in_directory(directory_path):
    total_lines = 0
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                total_lines += count_lines_in_file(file_path)
    return total_lines

training_lines = count_lines_in_directory(os.path.join(train_dir, "labels"))
validation_lines = count_lines_in_directory(os.path.join(valid_dir, "labels"))

print(f"Piscines entraînement: {training_lines}")
print(f"Piscines validation: {validation_lines}")

Piscines entraînement: 612
Piscines validation: 99


On déplace le cd dans le dir yolov5

In [2]:
%cd ../yolov5

c:\Users\rbubloz\Documents\Uni\MA_ISH\cours\02_traitim\seminaire\fedpool\yolov5


Définir la configuration pour les données et les hyperparamètres

In [3]:
import yaml
config = {'path': 'modele', # 'absolute/path/to/dir', https://stackoverflow.com/questions/66605322/yolo-v5-issue-exception-dataset-not-found-on-local-machine
         'train': 'modele/train',
         'val': 'modele/valid',
         'nc': 1,
         'names': ['swimming_pool']}
 
with open("data.yaml", "w") as file:
   yaml.dump(config, file, default_flow_style=False)

# Define the data
data_augm = {
    'lr0': 0.01,
    'lrf': 0.1,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 0.05,
    'cls': 0.3,
    'cls_pw': 1.0,
    'obj': 0.7,
    'obj_pw': 1.0,
    'iou_t': 0.2,
    'anchor_t': 4.0,
    'fl_gamma': 0.0,
    'hsv_h': 0,
    'hsv_s': 0,
    'hsv_v': 0,
    'degrees': 0.0,
    'translate': 0,
    'scale': 0,
    'shear': 0,
    'perspective': 0.0,
    'flipud': 0.5, # Flip up-down (prob)
    'fliplr': 0.5, # Flip left-right (prob)
    'mosaic': 0.0,
    'mixup': 0.0,
    'copy_paste': 0.0
}

# Write the data to the YAML file
with open('hyp_light_da.yaml', 'w') as file:
    yaml.dump(data_augm, file, default_flow_style=False)

Lancer (depuis une console python et depuis `/yolov5`) l'entraînement

`python train.py --img 1000 --batch 4 --epochs 15 --data data.yaml --weights yolov5m.pt --workers 4 --project fedpool --name fedpool_model --exist-ok --hyp hyp_light_da.yaml`

On trouve un aperçu des résultats dans `/fedpool_model`